In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import os 
import missingno as msno

from data_cleanup import *
from feature_selection import *
from model_ import *

from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as rmse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LassoLarsCV
from sklearn.ensemble import RandomForestRegressor

## Questions
Which country’s characteristics are good predictors of a country’s position on a corruption index?

If we manage to find characteristics and predict country’s position on a corruption index reasonably well, are there countries whose characteristics don’t correspond to their position on a corruption index?

If yes, does this hold across all corruption indexes, or do the lists of such countries vary from index to index?

## Dataset Loading
Our main dataset is the [QoG (Quality of Governence) Standard Time-Series Dataset](https://www.gu.se/en/quality-government/qog-data/data-downloads/standard-dataset) from the University of Gothenburg. It is a huge dataset with 2000 features that includes data from 1946 to 2021 for most countries. We chose it because we never really had to deal with dimensionality reduction yet and where interested in doing so. We additionally merge this dataset with the [ISO-3166-Countries with Regions](https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv) dataset to additionally obtain sub-region (f.e. Western Europe, South-East Asia, etc.) information, as we think the subregion of a country is very important for its corruption rankings. Due to this we are using the subregion to split the data in a stratified fashion into training/test-data.

For region merging manual patching was required as some countries that where included in the QoG dataset where not inside the ISO-3166 dataset. Additionally we opted to merge the Micronesia/Melanesia/Polynesia subregions into a new, combined region called 'Pacific Island' as the original subregions only had a small amount of entries.

In [ ]:
data_dir = 'data'
qog_dataset_filename = 'qog_std_ts_jan22.csv'
df = pd.read_csv(join(data_dir, qog_dataset_filename), low_memory=False)

df = merge_region(df)
display(df)


## Dataset Exploration

We can see that there are some features with a very small amount of non-null values. We can also see that most features are numerical values or might be encoded as numerical values. Checking this with the documentation of the dataset shows that this indeed seems to be the case as all categorical variables were encoded numerically.

In [ ]:
df.info(verbose=True, memory_usage='deep', show_counts=True)
df.describe()
df.dtypes.value_counts()

Next we are looking at the correlation between corruption indices that we decided to investigate more closely after our preliminary research.
We can see that most of them are highly correlated which makes sense as they generally try to estimate the same thing with slightly different approaches.

In [ ]:
corruption_col = ['bci_bci', 'ti_cpi', 'vdem_corr', 'vdem_execorr', 'vdem_jucorrdc', 'vdem_pubcorr', 'wbgi_cce', 'ti_cpi_om']
corruption_corr = df[corruption_col].corr()
mask = np.triu(np.ones_like(corruption_corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corruption_corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

As numbers don't tell the whole story we also look at pairplots between the different corruption indices. The main corruption indices (bci_bci, ti_cpi and wbgi_cce) seem to be really close in their estimates.

In [ ]:
sns.pairplot(df[corruption_col])

We are looking at missing values next. We can see that previously less data was available. The closer we are to recent times the more data was available. We can also see a somewhat regular-ish pattern of missing values across multiple years. This most likely are countries for which no data was gathered across multiple years (f.e. Western countries generally started collecting extensive data earlier while countries in sub-saharan africa where rather late). 

Due to changes in the methodology for ti_cpi (which we can see quite nicely in the plots) we have to handle this case in a more special case. Explanations for it follow in the next steps.

As we have 2000 features it doesn't really make sense to look at the distribution/value-ranges/missing values, ... etc for those, as this would take an unreasonable amount of time to do.

In [ ]:
# drop every row where none of the corruption data is available
df_any_corruption_info_available = df.dropna(subset=corruption_col, axis=0, how="all")

# drop every row where not every corruption data is available with special handling for ti_cpi
df_cpi_combined = df.copy()
df_cpi_combined['ti_cpi']=df['ti_cpi'].combine_first(df['ti_cpi_om'])
df_all_corruption_info_available = df_cpi_combined.dropna(subset=corruption_col, axis=0, how='any')


display(df_any_corruption_info_available.shape)
#11k to 1.7k rows
display(df_all_corruption_info_available.shape)

corruption_col_with_year = ['year', 'bci_bci', 'ti_cpi', 'ti_cpi_om', 'vdem_corr', 'vdem_execorr', 'vdem_jucorrdc', 'vdem_pubcorr', 'wbgi_cce']
ax = msno.matrix(df[corruption_col_with_year].sort_values(by='year'))
ax.set_title('Missing values from whole dataframe sorted by year (most recent being lowest)', fontsize=18)
ax = msno.matrix(df_any_corruption_info_available[corruption_col_with_year].sort_values(by='year'))
ax.set_title('Missing values from dataframe with all rows where any corruption info is available sorted by year (most recent being lowest)', fontsize=18)
# No surprises here
corruption_col_with_year.remove('ti_cpi_om')
ax = msno.matrix(df_all_corruption_info_available[corruption_col_with_year].sort_values(by='year'))
ax.set_title('Missing values from dataframe with all rows where all corruption info is available sorted by year (most recent being lowest)', fontsize=18)

Its also important to get an idea of the value range for each corruption metric.

In [ ]:
df[corruption_col].describe()

## Dataset preprocessing

In the next step we are dropping some features that were directly used by the corruption indices that we want to predict. If we would keep them our feature selection just would end up picking those features and we would only learn the exact same model that the institutions used to calculate their corruption indices. We opted for a smaller amount of corruption indices so that the scope of this exercise didn't get to big.

In [ ]:
# columns with corruption indices
corr_cols = ['ti_cpi', 'bci_bci', 'ti_cpi_om', 'wbgi_cce'] 
# columns with metadata
meta_cols = ['ccode', 'ccode_qog', 'ccodealp', 'ccodealp_year', 'ccodecow', 'cname', 'cname_qog', 'cname_year', 'version', 'year', 'region', 'sub-region']
df_reduced = drop_certain_columns(df, corr_cols, meta_cols)
display(df_reduced.shape)

The dataset contains data from 1960-2021. As most corruption indices only started to be available in the 1990s the data from the years before that are not that interesting(we only consider 'this' years data to calculate 'this' years result). Additionally we will also end up training one model for each corruption index and as we want to make comparisons between them we will have to use the exact same train and testset for all of them. Due to this we only keep the rows where values for all corruption indices are available. Unfortunately ti_cpi (Transparency international - Corruption Perception Index) drastically changed its methodology in 2012, which means that we have two variables for it ti_cpi and ti_cpi_om (old methodology), as they can't be directly compared. If we would restrict the data to only include the new methodology we would have a very small dataset. But the old methodology is not as interesting as it is a relative measure (relative to other countries in that year) that can't be directly compared across different years (so having the same ti_cpi_om-value in 1990 and 1995 could mean different things). 

Because of this we opted to keep rows where the union of ti_cpi and ti_cpi_om with the caveat that 
- those models can't be directly compared to the other models (bci_bci and wbgi_cce) because they don't have the same training/testdata
- the model for ti_cpi_om might not be that meaningful due to being a relative measure

In [ ]:
df_reduced = return_rows_where_all_corruption_data_is_available(df_reduced, corr_cols)
display(df_reduced.shape)

Next we drop all rows that have less than 10% non-nan values as they don't hold that much information anyway and because we are still left with a very large number of features.
Then we transform features that have less than 10 unique values in 2836 rows into categorical variables. This picks up a small amount of false positives but in a huge majority of cases this transformation was correct. Due to the high amount of features we didn't bother with doing it 'correctly' (which would mean by hand).

As we still have a very high amount of features we drop all columns that have any nan value so that we don't have to correctly handle them. We are left with 700 features.

In [ ]:
df_reduced = drop_columns_with_nan_values(df_reduced)
display(df_reduced.shape)
df_reduced = transform_to_categorical(df_reduced)
display(df_reduced.shape)

df_reduced.loc[:,['ti_cpi', 'ti_cpi_om']] = df_reduced.loc[:,['ti_cpi', 'ti_cpi_om']].replace(np.NaN, -5)
df_reduced = df_reduced.dropna(how='any', axis=1).copy()
df_reduced.loc[:,['ti_cpi', 'ti_cpi_om']] = df_reduced.loc[:,['ti_cpi', 'ti_cpi_om']].replace(-5, np.NaN)
display(df_reduced.shape)
df = df_reduced

### Feature selection

In [ ]:
df = drop_date_columns(df)

best_features_dict = {}
selected_features_dict = {}

for target_col in corr_cols:
    X_train, X_test, y_train, y_test = create_traintestsplit(df, corr_cols = corr_cols, meta_cols=meta_cols, target_col=target_col)
    
    best_features = pre_select(X_train, y_train)
    best_features_dict[target_col] = set(best_features)
    df_train = X_train[best_features].copy()
    df_train[target_col]=y_train
    mce = MultiCollinearityEliminator(df_train, target_col, 0.85)
    feaures_no_collinearity = list(mce.autoEliminateMulticollinearity().columns)
    feaures_no_collinearity.remove(target_col)
    selected_features_dict[target_col] = set(feaures_no_collinearity)


#selected_features_dict

In [ ]:
best_features_union=list(set.union(*list(best_features_dict.values())))
best_features_intersection=list(set.intersection(*list(best_features_dict.values())))

best_features_intersection

In [ ]:
selected_features_union=list(set.union(*list(selected_features_dict.values())))
selected_features_intersection=list(set.intersection(*list(selected_features_dict.values())))

selected_features_intersection

## Modeling

Try Lasso and Random Forest next. Train models for different feature configurations 

    - individual selected features for a particular index
    - union of all good features for all indices
    - intersection of all selected for features for all indices

As scores r2 and rmse are reported. The comparisons are based on r2-scores as they make the scores for different indices comparable.
    

### Lasso
The used library uses cross validation to determine a good value for alpha.

The following script trains for all target indices a Lasso model, then displays r2 score and feature importance information. 

In [ ]:
def lasso_info_script(features, name):
    lasso_bf = dict()

    df_score = pd.DataFrame(columns=['r2', 'rmse'], index=corr_cols)
    for target in corr_cols:
        if isinstance(features, dict):
            lasso_bf[target] = apply_lassocv(df, target, list(features[target]), corr_cols, meta_cols, fprint=False)
        else:
            lasso_bf[target] = apply_lassocv(df, target, features, corr_cols, meta_cols, fprint=False)
        df_score.loc[target,] = [lasso_bf[target]['r2'] ,lasso_bf[target]['rmse']]
    
    print('scores')
    display(df_score)

    # l_fi = [lasso_bf[target]['feat_importance'] for target in corr_cols]
    # df_fi = pd.concat(l_fi)

    # l_firk = [lasso_bf[target]['feat_importance_rank'] for target in corr_cols]
    # df_firk = pd.concat(l_firk)

    # print('feature importance')
    # display(df_fi)
    # df_fi.T.plot(kind='bar', figsize=(20,8))
    # print()

    # print('feature importance rank')
    # display(df_firk)
    # print()
    # file = os.path.join('pickle', name +'.obj')
    # f = open(file, 'wb')
    # pickle.dump(lasso_bf ,f)
    #f.close()     


First we apply the script for the individually selected features for each corruption index.

In [ ]:
lasso_info_script(selected_features_dict, 'lasso_selected_features_dict')

Next we use for all corruption indices the same set of features - the set of all as promising declared features.

In [ ]:
lasso_info_script(best_features_union, 'lasso_best_features_union')

In [ ]:
df.dtypes[best_features_union]
df.br_mon.describe()

Now we use only the features that are in all individually selected feature sets.

In [ ]:
lasso_info_script(selected_features_intersection, 'lasso_selected_features_intersection')

Findings:

- wbgi_rle (Rule of Law) is by far the most important feature in almost all configurations
- most indices behave similarly for the three feature set configuration but
- ti_cpi is most different: its score is very bad with the smallest feature set. Its most important feature is wbgi_pvs [Political Stability and Absence of Violence/Terrorism, Standard error] and not wbgi_rle
- vdem_jucorrdc is also effected more by different feature sets and its score is lower as well in general.
- all the other indices gain information slightly by more features but they do not rely too much on the chosen setups

### Random Forest

Next we do the same for a Random Forest Regressor. Here initially no cross validation is done. We just use a default setup at first.

In [ ]:
def rf_info_script(features, name):
    rf_bf = dict()

    df_score = pd.DataFrame(columns=['r2', 'rmse'], index=corr_cols)
    for target in corr_cols:
        if isinstance(features, dict):
            rf_bf[target] = apply_rf(df, target, list(features[target]), corr_cols, meta_cols, fprint=False)
        else:
            rf_bf[target] = apply_rf(df, target, features, corr_cols, meta_cols, fprint=False)
        df_score.loc[target,] = [rf_bf[target]['r2'] ,rf_bf[target]['rmse']]
    
    print('scores')
    display(df_score)

    # l_fi = [rf_bf[target]['feat_importance'] for target in corr_cols]
    # df_fi = pd.concat(l_fi)

    # l_firk = [rf_bf[target]['feat_importance_rank'] for target in corr_cols]
    # df_firk = pd.concat(l_firk)

    # print('feature importance')
    # display(df_fi)

    # df_fi.T.plot(kind='bar', figsize=(20,8))
    # print()

    # print('feature importance rank')
    # display(df_firk)
    # print()
    # file = os.path.join('pickle', name +'.obj')
    # f = open(file, 'wb')
    # pickle.dump(rf_bf ,f)
    #f.close()    



First we apply again the script for the individually selected features for each corruption index.

In [ ]:
rf_info_script(selected_features_dict, 'rf_selected_features_dict')

Next we use for all corruption indices the same set of features - the set of all as promising declared features.

In [ ]:
rf_info_script(best_features_union, 'rf_best_features_union')

Now we use only the features that are in all individually selected feature sets.

In [ ]:
rf_info_script(selected_features_intersection, 'rf_best_features_union')

The general picture of the results with Random Forest is not that different to the one with Lasso. Some differences are

- ti_cpi is predicted very well now both in comparison with Lasso and with all other indices
- HOWEVER, if only the minimal feature set is used ti_cpi is even worse than with Lasso
- for vdem_execorr the vdem_egal (Egalitarian component index) is the most important feature
- vdem_jucorr is now by far the most difficult to predict index
- although feature importance is not straight-forward comparable between Lasso (weight of coefficients) and Random Forest (Gini) it seems like Random Forst discriminates harder with regard to features

Random Forest performs either similarly or better for most setups / indices allthough no parameter optimization is done by now. So we continue with Random Forst and do hyperparameter optimization for some specific settings next to further optimize the results.

### Grid Search: Random Forest

With cross validation / hyperparameter grid search better parameters are determined. With those optimizations then again models are trained, then the test set is predicted and scores are evaluated.

The script defined below shows a similar report than above.

In [ ]:
def rf_gridsearch_info_script(features, name):
    rf_bf = dict()

    param_grid = {
        "randomforestregressor__max_depth": [2, 3, 5, 10, None],
        "randomforestregressor__min_samples_split": [2, 3, 5, 10],
        "randomforestregressor__max_features": ["log2", None]
        }

    df_score = pd.DataFrame(columns=['r2', 'rmse'], index=corr_cols)
    for target in corr_cols:
        if isinstance(features, dict):
            rf_bf[target] = apply_gridsearch_rf(df, target, list(features[target]), param_grid, corr_cols, meta_cols, fprint=False)
        else:
            rf_bf[target] = apply_gridsearch_rf(df, target, features, param_grid, corr_cols, meta_cols, fprint=False)
        df_score.loc[target,] = [rf_bf[target]['r2'] ,rf_bf[target]['rmse']]
    
    print('scores')
    display(df_score)

    # l_fi = [rf_bf[target]['feat_importance'] for target in corr_cols]
    # df_fi = pd.concat(l_fi)
    # rf_bf[target]
    # l_firk = [rf_bf[target]['feat_importance_rank'] for target in corr_cols]
    # df_firk = pd.concat(l_firk)
    # rf_bf[target]['params']
    # l_params = [rf_bf[target]['params'] for target in corr_cols]
    # df_params = pd.concat(l_params)

    # print('feature importance')
    # display(df_fi)

    # df_fi.T.plot(kind='bar', figsize=(20,8))
    # print()

    # print('feature importance rank')
    # display(df_firk)
    
    # file = os.path.join('pickle', name +'.obj')
    # f = open(file, 'wb')
    # pickle.dump(rf_bf ,f)
    #f.close()    



Now we only use for each index the individually selected feature set as we saw above that the results are comparable (so the feature selection process works adequately).

In [ ]:
rf_gridsearch_info_script(selected_features_dict, 'rf_grid_selected_features_dict')

For most indices the hyperparameter optimization does not seem to significantly improve the r2-scores. But for vdem_jucorrdc it seems to improve. For vdem_pubcorr and wbgi_cce the improvement is minor.

The feature importance (figure) changes a lot more. Here we see for all but bci_bci that relatively wbgi_rle is not as important anymore. This is most likely due to the max_samples_features being log2 now. One could argue if the original model where wbgi_rle is the main feature is simpler and from the same quality or on the other side that other features are also able to replace wbgi_rle when combined.

In [ ]:
rf_bf = rf_gridsearch_info_script(selected_features_union, 'rf_grid_selected_features_union')

In [ ]:

f = open('pickle/rf_grid_selected_features_union.obj', 'rb')
rf_bf = pickle.load(f)
f.close()

l_fi = [rf_bf[target]['feat_importance'] for target in corr_cols]
df_fi = pd.concat(l_fi)
l_firk = [rf_bf[target]['feat_importance_rank'] for target in corr_cols]
df_firk = pd.concat(l_firk)
l_params = [rf_bf[target]['params'] for target in corr_cols]
df_params = pd.concat(l_params)

print('feature importance')
display(df_fi)


df_sorted = df_fi.reindex(df_fi.mean().sort_values().index[::-1], axis=1)

display(df_sorted)
df_sorted.T.plot(kind='bar', figsize=(20,8))

df_fi.T.plot(kind='bar', figsize=(20,8))
print()

print('feature importance rank')
display(df_firk)
print()
print(df_sorted.columns)




df_fi = df_fi.reindex(df_fi.mean().sort_values(ascending=False).index, axis=1)
col_names = df_fi.columns
df_fi = df_fi.T.melt(
    ignore_index=False,
    value_vars = ['ti_cpi', 'bci_bci', 'ti_cpi_om', 'wbgi_cce'],
    value_name = 'feature_importance'
).reset_index().rename(columns={'index': 'feature', 'variable': 'corruption_index'})

plt.rcdefaults()
font = {'family' : 'normal',
    'size'   : 14}

plt.rc('font', **font)
plt.figure(figsize=(20,8))
sns.barplot(df_fi, x='feature',  y='feature_importance', hue='corruption_index', palette='magma', width=0.6)
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.grid()